In [1]:
import numpy as np
from scipy.optimize import minimize
import math

In [2]:
def main_function(alpha, Mat):
    
    return 0.5 * np.dot(alpha, np.dot(Mat, alpha)) - np.sum(alpha)

In [3]:
def jac(alpha, Mat):
    
    return np.dot(alpha.T, Mat) - np.ones(alpha.shape[0])

In [4]:
def dual_svm(train_x, train_y, C):
    
    num_data=train_x.shape[0]
    
    # set parameters
    x0 = np.random.rand(num_data)
    bnds = [(0, C)] * num_data
    Mat = np.zeros((num_data, num_data))
    
    def Mat_func(train_x,train_y):
        for i in range(num_data):
            for j in range(num_data):
                Mat[i,j]=np.dot(train_x[i],train_x[j])*train_y[i]*train_y[j]
        return Mat
    
    Mat=Mat_func(train_x,train_y)
    # optimize
    res = minimize(main_function, x0, args=(Mat,), method='L-BFGS-B', jac=jac, bounds=bnds)

    # recover w, b
    w = np.sum([res.x[i] * train_y[i] * train_x[i,:] for i in range(train_x.shape[0])], axis=0)
    b = np.mean(train_y - np.dot(train_x, w))

    return w, b

In [5]:
def dual_test(x,y,w,b):
    
    error=0
    num_data=x.shape[0]
    for i in range(num_data):
        tem=np.sign(w.dot(x[i]+b))
        if tem!=y[i]:
            error+=1
    return error/num_data

In [6]:
train_data = []
with open('train.csv', 'r') as f:
    for term in f:
        train_data.append(term.strip().split(','))
        
test_data= []
with open('test.csv', 'r') as f:
    for term in f:
        test_data.append(term.strip().split(','))

In [7]:
train_data = np.array(train_data, dtype='float64')
test_data = np.array(test_data, dtype='float64')

In [8]:
train_x= train_data[:, :-1]
train_y = train_data[:, -1].astype(int)
# convert y label with -1,1
train_y[train_y == 0] = -1  

In [9]:
test_x= test_data[:, :-1]
test_y = test_data[:, -1].astype(int)
# convert y label with -1,1
test_y[test_y == 0] = -1 

In [10]:
C_set =  [100.0/873, 500.0/873, 700.0/873]

In [11]:
#Q3.a
for c in C_set: 
    w,b=dual_svm(train_x,train_y,C=c)
    print('C:', c,'weights:', w, 'bias:', b)
    #train_error=dual_test(train_x,train_y,w,b)
    #test_error=dual_test(test_x,test_y,w,b)
    #print('training error:', train_error, 'test error', test_error)

C: 0.1145475372279496 weights: [-1.2683416  -0.67324123 -0.74689017 -0.25551805] bias: 2.4765196654753145
C: 0.572737686139748 weights: [-1.69303249 -0.70342801 -0.77135958 -0.15729592] bias: 2.8780229871343943
C: 0.8018327605956472 weights: [-1.7978385  -0.86819483 -1.08485354 -0.4180363 ] bias: 3.35720492782103
